In [6]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig

In [39]:
df = pd.read_csv("../../db/data/merge/tertiary/idxs_.csv",index_col=0)
df_logs = pd.read_csv("../../db/data/merge/tertiary/logs_.csv",index_col=0)

## *Exponential Moving Average*

In [25]:
def ema(df, window=14):
    """
    Exponential movint average
    """
    data = pd.DataFrame([])
    
    for currency in df.columns:
        data[f'{currency}_ema'] = df[currency].ewm(alpha=1/window, adjust=False).mean()
        
    data.index = df.index
    
    return data

## *Highpass Filter*

In [104]:
def highpass_filter(df, order=5, cutoff=0.2):
    """Highpass filter """
    
    data = pd.DataFrame([])
    
    b, a = sig.butter(N=order, Wn=cutoff, btype='highpass', analog=False)
    
    for currency in df.columns:
        data[f'{currency}_high'] = sig.lfilter(b, a, df[currency])
    
    data.index = df.index
    
    return data

## *Lowpass Filter*

In [106]:
def lowpass_filter(df, order=8, cutoff=0.2):
    """Lowpass filter """
    data = pd.DataFrame([])
    
    # filter
    b, a = sig.butter(N=order, Wn=cutoff, btype='lowpass', analog=False)
    
    for currency in df.columns:
        data[f'{currency}_low'] = sig.lfilter(b, a, df[currency])
    
    data.index = df.index
    
    return data


## *Relative Strenght Index*

In [1]:
def rsi(df, periods = 240):
    """Returns a series with the rsi """
    data = pd.DataFrame([])
    for currency in df.columns:
        delta = df[currency].diff()
        up = delta.clip(lower=0)
        down = -1*delta.clip(upper=0)
        
        ema_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        ema_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
        rs = ema_up / ema_down
        rsi = 100 - (100/(1 + rs))
        
        data[f'{currency}_rsi'] = rsi
        
    return data

## *Correlation*

In [8]:

SYMBOLS = [ 'AUD_CAD', 'AUD_CHF', 'AUD_JPY', 'AUD_NZD', 'AUD_USD', 
            'CAD_CHF', 'CAD_JPY', 'CHF_JPY', 'EUR_AUD', 'EUR_CAD', 
            'EUR_CHF', 'EUR_GBP', 'EUR_JPY', 'EUR_NZD', 'EUR_USD', 
            'GBP_AUD', 'GBP_CAD', 'GBP_CHF', 'GBP_JPY', 'GBP_NZD', 
            'GBP_USD', 'NZD_CAD', 'NZD_CHF', 'NZD_JPY', 'NZD_USD', 
            'USD_CAD', 'USD_CHF', 'USD_JPY'
          ]

In [55]:
def correlations(df, rate=120):
    data = pd.DataFrame([])
    for sym in SYMBOLS:
        data[f'{sym[:3]}_{sym[4:]}_corr'] = df[f'{sym[:3]}'].rolling(rate).corr(df[f'{sym[4:]}'])
    return data
        

In [56]:
cor = correlations(df_logs)


In [58]:
cor.iloc[200:300]

,AUD_CAD_corr,AUD_CHF_corr,AUD_JPY_corr,AUD_NZD_corr,AUD_USD_corr,CAD_CHF_corr,CAD_JPY_corr,CHF_JPY_corr,EUR_AUD_corr,EUR_CAD_corr,...,GBP_JPY_corr,GBP_NZD_corr,GBP_USD_corr,NZD_CAD_corr,NZD_CHF_corr,NZD_JPY_corr,NZD_USD_corr,USD_CAD_corr,USD_CHF_corr,USD_JPY_corr
2005-01-13 08:00:00+00:00,-0.211780,-0.183586,-0.357099,0.536238,-0.360885,-0.163294,-0.057060,-0.311144,-0.212260,-0.243550,...,-0.019732,-0.441088,0.229231,-0.179159,-0.147417,-0.198648,-0.456685,-0.141329,-0.425651,0.056451
2005-01-13 09:00:00+00:00,-0.211732,-0.183705,-0.356668,0.536117,-0.361678,-0.163593,-0.058617,-0.311388,-0.212364,-0.243953,...,-0.023690,-0.448551,0.260160,-0.181568,-0.149404,-0.195883,-0.461241,-0.142483,-0.428128,0.055722
2005-01-13 10:00:00+00:00,-0.207973,-0.184877,-0.353051,0.535736,-0.359124,-0.158832,-0.065889,-0.316129,-0.218152,-0.233484,...,0.004442,-0.442190,0.240656,-0.179798,-0.149484,-0.194374,-0.458920,-0.143087,-0.420266,0.043672
2005-01-13 11:00:00+00:00,-0.214543,-0.176004,-0.342592,0.519475,-0.354018,-0.158536,-0.065356,-0.323707,-0.195120,-0.235588,...,-0.005773,-0.427382,0.234827,-0.184480,-0.140484,-0.182254,-0.456634,-0.142364,-0.425631,0.038739
2005-01-13 12:00:00+00:00,-0.207536,-0.183714,-0.348166,0.537446,-0.354517,-0.156603,-0.061344,-0.326677,-0.198662,-0.235270,...,-0.010393,-0.423624,0.236473,-0.203251,-0.124213,-0.177091,-0.462400,-0.142683,-0.425624,0.039056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2005-01-19 07:00:00+00:00,-0.219788,-0.269265,-0.232658,0.421041,-0.335409,-0.028547,-0.041596,-0.323314,-0.290910,-0.170476,...,-0.394170,-0.205010,0.102457,-0.100522,-0.225485,-0.327358,-0.372765,-0.361350,-0.349235,0.197493
2005-01-19 08:00:00+00:00,-0.212970,-0.239836,-0.256698,0.409301,-0.360602,-0.028112,-0.037687,-0.349815,-0.236176,-0.161225,...,-0.404331,-0.222361,0.099485,-0.106868,-0.239156,-0.294764,-0.346992,-0.350390,-0.369370,0.220403
2005-01-19 09:00:00+00:00,-0.208154,-0.241699,-0.263357,0.410619,-0.358210,-0.030162,-0.044839,-0.343809,-0.237241,-0.162563,...,-0.396401,-0.220059,0.098165,-0.104441,-0.240988,-0.299726,-0.345585,-0.348462,-0.369665,0.217096
2005-01-19 10:00:00+00:00,-0.190396,-0.256407,-0.276698,0.425895,-0.350384,-0.037793,-0.052936,-0.331327,-0.226393,-0.161179,...,-0.348276,-0.258275,0.099548,-0.087473,-0.254211,-0.311693,-0.340792,-0.350940,-0.365691,0.219284
